In [8]:
from ioh import get_problem, ProblemClass
from ioh import logger
import sys
import numpy as np
import time

In [9]:
# Declaration of problems to be tested.
# We obtain an interface of the OneMax problem here.
dimension = 50

In [22]:
"""
1 (fid) : The funciton ID of the problem in the problem suite. OneMax is 1 defined within the PBO class. 2 would correspond to another problem.
dimension : The dimension of the problem, which we have set to 50.
instance: In benchmarking libraries, problems often have multiple instances. These instances may vary slightly (e.g., different random noise, shifts, etc.) 
            to allow algorithms to be tested on a variety of conditions.
om(x) return the fitness value of 'x'
"""
om = get_problem(1, dimension=dimension, instance=1, problem_class=ProblemClass.PBO)
# We know the optimum of onemax
optimum = dimension

In [23]:
# Create default logger compatible with IOHanalyzer
# `root` indicates where the output files are stored.
# `folder_name` is the name of the folder containing all output. You should compress the folder 'run' and upload it to IOHanalyzer.
l = logger.Analyzer(root="data", 
    folder_name="run", 
    algorithm_name="genetic_algorithm", 
    algorithm_info="The lab session of the evolutionary algorithm course in LIACS")

In [24]:
om.attach_logger(l)

In [40]:
om.meta_data.name


'OneMax'

In [14]:
# Parameters setting
pop_size = 100
tournament_k = 1
mutation_rate = 0.02
crossover_probability = 0.6

In [15]:
a = np.random.rand()
a

0.7537165442981083

In [16]:
# Uniform Crossover
def crossover(p1, p2):
    if np.random.rand() < crossover_probability:
        for i in range(len(p1)):
            if np.random.rand() < 0.5:
                v = p1[i]
                p1[i] = p2[i]
                p2[i] = v
    return p1,p2

In [17]:
# Standard bit mutation using mutation rate p
def mutation(p):
    for i in range(len(p)):
        if np.random.rand() < mutation_rate:
            p[i] = abs(p[i] - 1)
            
    return p

In [18]:
# Using the Fitness proportional selection
def mating_seletion(parent, parent_f) :    
    # c = lowest score of parent_f
    # sum total fitness score,
    # p = f[i] - c / total - c * mew (avg score?) -> check lecture
    # mew is population size
    
    c = min(parent_f) - 0.001
    S_f = sum(parent_f)
    probabilities = []
    
    for i in range(len(parent)):
        p = (parent_f[i] - c) / (S_f - (c * len(parent)))
        probabilities.append(p)
        
    # new parents
    offspring = []
    for i in range(len(parent)):
        pick = np.random.choice(parent, p=probabilities)
        offspring.append(pick)
    
    return offspring

In [19]:
def genetic_algorithm(func, budget = None):
    
    # budget of each run: 10000
    if budget is None:
        budget = 10000
    
    # f_opt : Optimal function value.
    # x_opt : Optimal solution.
    f_opt = sys.float_info.min
    x_opt = None
    
    # parent : A list that holds the binary strings representing potential solutions or individuals in the current population.
    # parent_f : A list that holds the fitness values corresponding to each individual in the parent list.
    parent = []
    parent_f = []
    for i in range(pop_size):

        # Initialization
        parent.append(np.random.randint(2, size = func.meta_data.n_variables))
        parent_f.append(func(parent[i]))
        budget = budget - 1

    while (f_opt < optimum and budget > 0):

        # Perform mating selection, crossover, and mutation to generate offspring
        offspring = mating_seletion(parent, parent_f)
        
        
        for p in range(0, (len(parent)), 2):
            offspring[p], offspring[p+ 1] = crossover(offspring[p], offspring[p+ 1])
        
        for i in range(len(offspring)):
            offspring[i] = mutation(offspring[i])
        
        parent = offspring    
        parent_f = [func(offspring[i]) for i in offspring]    
        
        best_idx = np.argmax(parent_f)
        x_opt = parent(best_idx)
        f_opt = parent_f(best_idx)
    # ioh function, to reset the recording status of the function.
    func.reset()
    print(f_opt,x_opt)
    return f_opt, x_opt

In [20]:
def main():
    # We run the algorithm 20 independent times.
    for _ in range(20):
        genetic_algorithm(om)

In [21]:
if __name__ == '__main__':
  start = time.time()
  main()
  end = time.time()
  print("The program takes %s seconds" % (end-start))

ValueError: a must be 1-dimensional